In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

### Datos 2023-2024

In [5]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('Totalizadores Planta de Cerveza 2023_2024.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2023_2024 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2023_2024[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

--- Resumen de hojas y columnas ---
                         hoja  filas  columnas  \
0             Consolidado KPI  12010       125   
1                       Metas     48        57   
2      Consolidado Produccion  12011        19   
3    Totalizadores Produccion  12009        41   
4              Consolidado EE  12011        24   
5       Totalizadores Energia  12009        54   
6            Consolidado Agua  12011        24   
7          Totalizadores Agua  12009        44   
8        Consolidado GasVapor  12011        20   
9   Totalizadores Gas y Vapor  12009        24   
10           Consolidado Aire  12011        14   
11         Totalizadores Aire  12009        12   
12          Totalizadores CO2  12009         9   
13    Totalizadores Efluentes  12009         9   
14       Totalizadores Glicol  12009         8   
15            Seguimiento Dia  12009         4   
16                   Auxiliar  12011        38   
17          Kw Frio  Hl Mosto  12372        15   

             

In [6]:
nombre_hoja_para_ver = 'Consolidado KPI'

if nombre_hoja_para_ver in dfs_2023_2024:
    
    print(f"  Mostrando la hoja: {nombre_hoja_para_ver}")
    
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 3 FILAS (.head()) ---")
    print(dfs_2023_2024[nombre_hoja_para_ver].head(3).to_string())
    
    print("\n\n--- ÚLTIMAS 3 FILAS (.tail()) ---")
    print(dfs_2023_2024[nombre_hoja_para_ver].tail(3).to_string())

else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs_2023_2024.keys()))

  Mostrando la hoja: Consolidado KPI

--- PRIMERAS 3 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl  Aire L4 / Hl  Aire L5 / Hl  Aire Se

In [7]:
print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs_2023_2024.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_lista = sorted(list(set(todos_los_dias) - set(dias_con_23_59)))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_lista = sorted(list(dias_con_horas_faltantes_sr.index))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_lista)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59                              dias_con_horas_faltantes
hoja                                                                                                                                                         
Consolidado KPI            2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)  5 días (Ej: 2023-01-17, 2023-04-13, 2023-04-19, ...)
Metas                      2021-01-01  2024-12-01                                 N/A (Hoja no horaria)                                 N/A (Hoja no horaria)
Consolidado Produccion     2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)  5 días (Ej: 2023-01-17, 2023-04-13, 2023-04-19, ...)
Totalizadores Produccion   2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)  5 días (Ej: 2023-01-17, 2023-04-13, 2023-04-19, ...)
Consolidado 

In [8]:
dias_sin_23_59_lista

[datetime.date(2023, 2, 28),
 datetime.date(2023, 4, 13),
 datetime.date(2023, 4, 19),
 datetime.date(2024, 10, 26)]

In [9]:
if 'dfs_2023_2024' not in globals() or not isinstance(dfs_2023_2024, dict) or not dfs_2023_2024:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs_2023_2024.keys()):
        df = dfs_2023_2024[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs_2023_2024.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2023-01-01  2024-04-26                                                 Ninguno
Consolidado Agua           2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado Aire           2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado EE             2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado GasVapor       2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado KPI            2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado Produccion     2023-01-01  2024-10-26  188 día

In [10]:
dias_faltantes_lista

[datetime.date(2023, 3, 31),
 datetime.date(2023, 5, 31),
 datetime.date(2023, 10, 31),
 datetime.date(2023, 12, 31),
 datetime.date(2024, 1, 1),
 datetime.date(2024, 1, 2),
 datetime.date(2024, 1, 3),
 datetime.date(2024, 1, 4),
 datetime.date(2024, 1, 5),
 datetime.date(2024, 1, 6),
 datetime.date(2024, 1, 7),
 datetime.date(2024, 1, 8),
 datetime.date(2024, 1, 9),
 datetime.date(2024, 1, 10),
 datetime.date(2024, 1, 11),
 datetime.date(2024, 1, 12),
 datetime.date(2024, 1, 13),
 datetime.date(2024, 1, 14),
 datetime.date(2024, 1, 15),
 datetime.date(2024, 1, 16),
 datetime.date(2024, 1, 17),
 datetime.date(2024, 1, 18),
 datetime.date(2024, 1, 19),
 datetime.date(2024, 1, 20),
 datetime.date(2024, 1, 21),
 datetime.date(2024, 1, 22),
 datetime.date(2024, 1, 23),
 datetime.date(2024, 1, 24),
 datetime.date(2024, 1, 25),
 datetime.date(2024, 1, 26),
 datetime.date(2024, 1, 27),
 datetime.date(2024, 1, 28),
 datetime.date(2024, 1, 29),
 datetime.date(2024, 1, 30),
 datetime.date(2024, 

Una vez cargada toda la hoja de datos del 2023/2024 en un diccionario, visto los días faltantes y los días en los que no se cargó la última hora (23:59), vamos a crear un diccionario con todos los df con las filas que tengan la última hora de cada día ordenado por orden cronológico. 

In [11]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def _to_date(x):
    try:
        return pd.to_datetime(x, errors="coerce").date()
    except Exception:
        return pd.NaT

def _to_minutes(x):
    if pd.isna(x):
        return -1
    ts = pd.to_datetime(x, errors="coerce")
    if pd.notna(ts):
        return int(ts.hour) * 60 + int(ts.minute)
    try:
        h = int(float(str(x).replace(",", ".")))
        if 0 <= h <= 23:
            return h * 60
    except Exception:
        pass
    return -1

dfs_23_24 = {}
hojas_saltadas = []

for hoja, df in dfs_2023_2024.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)

    # Filtramos filas sin día y agregamos orden determinístico
    tmp = tmp.dropna(subset=["_dia"]).copy()
    if tmp.empty:
        hojas_saltadas.append((hoja, "Sin días válidos"))
        continue

    tmp["_ord"] = np.arange(len(tmp))  # <- evita usar el índice en sort_values

    # Orden por día, hora (minutos) y orden original
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # Última fila por día (la mayor "_mins"; si empata, la última por "_ord")
    ultimas = tmp.groupby("_dia", as_index=False, sort=True).tail(1)

    # Limpieza de columnas auxiliares y orden final
    ultimas = ultimas.drop(columns=["_dia", "_mins", "_ord"]).sort_values(DAY_COL).reset_index(drop=True)

    dfs_23_24[hoja] = ultimas

In [23]:
dfk = dfs_23_24["Consolidado EE"].copy()

dfk["_hora_dt"] = pd.to_datetime(dfk["HORA"], errors="coerce")

mask_no_2359 = ~(
    (dfk["_hora_dt"].dt.hour == 23) &
    (dfk["_hora_dt"].dt.minute == 59))

df_no_2359 = dfk[mask_no_2359].drop(columns=["_hora_dt"])

# Ver resultados
print(len(df_no_2359), "filas con última hora distinta de 23:59")
print(df_no_2359["HORA"].value_counts(dropna=False).head())
df_no_2359.head(4)

4 filas con última hora distinta de 23:59
HORA
23:00:00    1
19:00:00    1
16:00:00    1
07:00:00    1
Name: count, dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14256\3529502605.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfk["_hora_dt"] = pd.to_datetime(dfk["HORA"], errors="coerce")


,DIA,HORA,Planta (Kw),Elaboracion (Kw),Bodega (Kw),Cocina (Kw),Envasado (Kw),Linea 2 (Kw),Linea 3 (Kw),Linea 4 (Kw),Servicios (Kw),Sala Maq (Kw),Aire (Kw),Calderas (Kw),Efluentes (Kw),Frio (Kw),Pta Agua / Eflu (Kw),Prod Agua (Kw),Resto Serv (Kw),Restos Planta (Kw),KW Gral Planta,KW CO2,Fecha/Hora,Kw de Frio
58,2023-02-28,23:00:00,55102.470000,5783.0,4869.0,1277.0,17587.0,5315.470000,7026.0,7692.0,27452.0,16937.0,5429.0,267.0,1150.0,15279.0,1475.0,280.0,4664.0,4280.470000,56732.0,383.0,2023-08-28 23:00:00,15279.0
101,2023-04-13,19:00:00,30060.180000,5109.0,4020.0,1844.0,442.0,643.180000,549.0,0.0,22522.0,17006.0,3512.0,212.0,529.0,14315.0,701.0,141.0,3285.0,1987.180000,31193.0,528.0,2023-10-13 19:00:00,14315.0
107,2023-04-19,16:00:00,44194.720000,4184.0,3156.0,1329.0,13399.0,5103.220000,5115.0,5222.0,23010.0,15840.0,4861.0,313.0,543.0,13439.0,869.0,287.0,3393.0,3601.720000,45310.0,174.0,2023-10-19 16:00:00,13439.0
476,2024-10-26,07:00:00,8987.480469,842.0,973.0,76.0,1471.0,2104.980469,151.0,0.0,5521.0,3193.0,737.0,69.0,145.0,3124.0,181.0,22.0,1339.0,1153.480469,9467.0,85.0,2024-04-26 07:00:00,3124.0


Vamos a eliminar el último día ya que no sirve de nada y también interpolar los 5 días faltantes:

In [27]:
for nombre, df in dfs_23_24.items():
    if not df.empty:                      # evita error si alguna hoja está vacía
        dfs_23_24[nombre] = df.iloc[:-1].reset_index(drop=True)

In [28]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def completar_e_interpolar_diario(df, day_col=DAY_COL, hour_col=HOUR_COL, hora_por_defecto="23:59:00"):
    g = df.copy()

    # --- fecha como datetime (normalizada al día) ---
    g["_fecha"] = pd.to_datetime(g[day_col], errors="coerce", dayfirst=True).dt.normalize()
    g = g.dropna(subset=["_fecha"]).sort_values("_fecha").drop_duplicates("_fecha", keep="last")

    # --- índice continuo día a día (agrega los días faltantes) ---
    idx_full = pd.date_range(g["_fecha"].min(), g["_fecha"].max(), freq="D")
    g = g.set_index("_fecha").reindex(idx_full)

    # --- reconstruir columnas de fecha/hora ---
    g[day_col] = g.index.date
    if hour_col in g.columns:
        g[hour_col] = g[hour_col].fillna(hora_por_defecto)
    else:
        g[hour_col] = hora_por_defecto

    # --- interpolación SOLO en columnas numéricas ---
    num_cols = g.select_dtypes(include="number").columns
    if len(num_cols):
        # usa el índice temporal para interpolar; luego rellena bordes
        g[num_cols] = g[num_cols].interpolate(method="time").ffill().bfill()

    return g.reset_index(drop=True)

# Aplicarlo a TODO el diccionario (una hoja por vez)
for nombre, df in dfs_23_24.items():
    dfs_23_24[nombre] = completar_e_interpolar_diario(df)

In [37]:
DAY_COL = "DIA"
inicio  = pd.Timestamp("2023-12-31")
fin     = pd.Timestamp("2024-06-30")

for nombre, df in dfs_23_24.items():
    if DAY_COL not in df.columns or df.empty:
        continue

    # Normalizar a fecha y construir máscara para CONSERVAR lo que queda fuera del rango
    fechas = pd.to_datetime(df[DAY_COL], dayfirst=True, errors="coerce").dt.normalize()

    # Rango INCLUSIVO: elimina 31/12/2023 ... 30/06/2024
    mask_keep = (fechas < inicio) | (fechas > fin) | fechas.isna()

    dfs_23_24[nombre] = df.loc[mask_keep].reset_index(drop=True)

### Datos 2022-2023

Repetimos todo el proceso para los datos del 2022-2023

In [30]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('Totalizadores Planta de Cerveza - 2022_2023.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2022_2023 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2022_2023[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

--- Resumen de hojas y columnas ---
                         hoja  filas  columnas                                                                                                                                                                                         nombres_columnas
0             Consolidado KPI  15317       123  DIA, HORA, EE Planta / Hl, EE Elaboracion / Hl, EE Bodega / Hl, EE Cocina / Hl, EE Envasado / Hl, EE Linea 2 / Hl, EE Linea 3 / Hl, EE Linea 4 / Hl, EE Linea 5 / Hl, EE Servicios / Hl, EE Sala Maq...
1                       Metas     36        57  Mes / Año, Año + Mes, Agua Planta, EE Planta, ET Planta, Aire Planta, Unnamed: 6, Meta Agua Elab, Meta Agua Bodega, Meta Agua Cocina, Meta Agua Envas, Meta Agua Linea 2, Meta Agua Linea 3, Meta Ag...
2      Consolidado Produccion  15450        14  DIA, HORA, Hl de Mosto, Hl Cerveza Cocina, Hl Producido Bodega, Hl Cerveza Filtrada, Hl Cerveza Envasada, Hl Cerveza L2, Hl Cerveza L3, Hl Cerveza L4, Hl Cerveza L5, Cocimi

In [32]:
nombre_hoja_para_ver = 'Consolidado KPI'

if nombre_hoja_para_ver in dfs_2022_2023:
    
    print(f"  Mostrando la hoja: {nombre_hoja_para_ver}")
    
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 3 FILAS (.head()) ---")
    print(dfs_2022_2023[nombre_hoja_para_ver].head(3).to_string())
    
    print("\n\n--- ÚLTIMAS 3 FILAS (.tail()) ---")
    print(dfs_2022_2023[nombre_hoja_para_ver].tail(3).to_string())

else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs_2022_2023.keys()))

  Mostrando la hoja: Consolidado KPI

--- PRIMERAS 3 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl  Aire L4 / Hl  Aire L5 / Hl  Aire Se

In [33]:
print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs_2022_2023.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_lista = sorted(list(set(todos_los_dias) - set(dias_con_23_59)))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_lista = sorted(list(dias_con_horas_faltantes_sr.index))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_lista)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59                              dias_con_horas_faltantes
hoja                                                                                                                                                         
Consolidado KPI            2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  5 días (Ej: 2022-07-01, 2022-07-13, 2022-11-01, ...)
Metas                      2021-01-01  2023-12-01                                 N/A (Hoja no horaria)                                 N/A (Hoja no horaria)
Consolidado Produccion     2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  5 días (Ej: 2022-07-01, 2022-07-13, 2022-11-01, ...)
Totalizadores Produccion   2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  5 días (Ej: 2022-07-01, 2022-07-13, 2022-11-01, ...)
Consolidado 

[datetime.date(2022, 3, 2),
 datetime.date(2022, 7, 13),
 datetime.date(2023, 2, 28),
 datetime.date(2023, 3, 6)]

In [35]:
if 'dfs_2022_2023' not in globals() or not isinstance(dfs_2022_2023, dict) or not dfs_2022_2023:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs_2022_2023.keys()):
        df = dfs_2022_2023[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs_2022_2023.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado Agua           2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado Aire           2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado EE             2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado GasVapor       2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado KPI            2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado Produccion     2022-01-01  2023-12-30  121 día

In [36]:
dias_faltantes_lista

[datetime.date(2022, 3, 31),
 datetime.date(2022, 5, 31),
 datetime.date(2022, 10, 31),
 datetime.date(2022, 12, 31),
 datetime.date(2023, 3, 7),
 datetime.date(2023, 3, 8),
 datetime.date(2023, 3, 9),
 datetime.date(2023, 3, 10),
 datetime.date(2023, 3, 11),
 datetime.date(2023, 3, 12),
 datetime.date(2023, 3, 13),
 datetime.date(2023, 3, 14),
 datetime.date(2023, 3, 15),
 datetime.date(2023, 3, 16),
 datetime.date(2023, 3, 17),
 datetime.date(2023, 3, 18),
 datetime.date(2023, 3, 19),
 datetime.date(2023, 3, 20),
 datetime.date(2023, 3, 21),
 datetime.date(2023, 3, 22),
 datetime.date(2023, 3, 23),
 datetime.date(2023, 3, 24),
 datetime.date(2023, 3, 25),
 datetime.date(2023, 3, 26),
 datetime.date(2023, 3, 27),
 datetime.date(2023, 3, 28),
 datetime.date(2023, 3, 29),
 datetime.date(2023, 3, 30),
 datetime.date(2023, 3, 31),
 datetime.date(2023, 4, 1),
 datetime.date(2023, 4, 2),
 datetime.date(2023, 4, 3),
 datetime.date(2023, 4, 4),
 datetime.date(2023, 4, 5),
 datetime.date(2023,

In [42]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

dfs_22_23 = {}
hojas_saltadas = []

for hoja, df in dfs_2022_2023.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)

    # Filtramos filas sin día y agregamos orden determinístico
    tmp = tmp.dropna(subset=["_dia"]).copy()
    if tmp.empty:
        hojas_saltadas.append((hoja, "Sin días válidos"))
        continue

    tmp["_ord"] = np.arange(len(tmp))  # <- evita usar el índice en sort_values

    # Orden por día, hora (minutos) y orden original
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # Última fila por día (la mayor "_mins"; si empata, la última por "_ord")
    ultimas = tmp.groupby("_dia", as_index=False, sort=True).tail(1)

    # Limpieza de columnas auxiliares y orden final
    ultimas = ultimas.drop(columns=["_dia", "_mins", "_ord"]).sort_values(DAY_COL).reset_index(drop=True)

    dfs_22_23[hoja] = ultimas

In [43]:
dfk = dfs_22_23["Consolidado EE"].copy()

dfk["_hora_dt"] = pd.to_datetime(dfk["HORA"], errors="coerce")

mask_no_2359 = ~(
    (dfk["_hora_dt"].dt.hour == 23) &
    (dfk["_hora_dt"].dt.minute == 59))

df_no_2359 = dfk[mask_no_2359].drop(columns=["_hora_dt"])

# Ver resultados
print(len(df_no_2359), "filas con última hora distinta de 23:59")
print(df_no_2359["HORA"].value_counts(dropna=False).head())
df_no_2359.head(4)

4 filas con última hora distinta de 23:59
HORA
23:00:00    3
08:00:00    1
Name: count, dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14256\2120028945.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfk["_hora_dt"] = pd.to_datetime(dfk["HORA"], errors="coerce")


,DIA,HORA,Planta (Kw),Elaboracion (Kw),Bodega (Kw),Cocina (Kw),Envasado (Kw),Linea 2 (Kw),Linea 3 (Kw),Linea 4 (Kw),Servicios (Kw),Sala Maq (Kw),Aire (Kw),Calderas (Kw),Efluentes (Kw),Frio (Kw),Pta Agua / Eflu (Kw),Prod Agua (Kw),Resto Serv (Kw),Restos Planta (Kw),KW Gral Planta,KW CO2,Fecha/Hora,Kw de Frio
60,2022-03-02,23:00:00,50076.11,7775.0,6118.0,2047.0,9258.0,2034.86,1766.0,6652.0,31239.0,20287.0,5390.0,396.0,1029.0,19294.0,1548.0,459.0,4078.0,1804.11,51549.0,593.0,2022-09-02 23:00:00,19294.0
191,2022-07-13,23:00:00,72768.88,6938.5,5845.5,1901.0,17542.0,6024.13,6683.0,7423.0,49265.0,37052.0,7059.0,827.0,781.0,33704.0,1809.0,920.0,4594.0,-976.62,76246.0,1380.0,2022-01-13 23:00:00,33704.0
419,2023-02-28,23:00:00,55102.47,5783.0,4869.0,1277.0,17587.0,5315.47,7026.0,7692.0,27452.0,16937.0,5429.0,267.0,1150.0,15279.0,1475.0,280.0,4664.0,4280.47,56732.0,383.0,2023-08-28 23:00:00,15279.0
425,2023-03-06,08:00:00,20527.51,1663.0,1669.0,109.0,6352.0,1989.01,2561.0,2729.0,10648.0,6671.0,1851.0,0.0,358.0,6595.0,412.0,32.0,1394.0,1864.51,21070.0,418.0,2023-09-06 08:00:00,6595.0


In [44]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

# Aplicarlo a TODO el diccionario (una hoja por vez)
for nombre, df in dfs_22_23.items():
    dfs_22_23[nombre] = completar_e_interpolar_diario(df)

In [45]:
DAY_COL = "DIA"
inicio  = pd.Timestamp("2023-03-07")
fin     = pd.Timestamp("2023-06-30")

for nombre, df in dfs_22_23.items():
    if DAY_COL not in df.columns or df.empty:
        continue

    # Normalizar a fecha y construir máscara para CONSERVAR lo que queda fuera del rango
    fechas = pd.to_datetime(df[DAY_COL], dayfirst=True, errors="coerce").dt.normalize()

    # Rango INCLUSIVO: elimina 31/12/2023 ... 30/06/2024
    mask_keep = (fechas < inicio) | (fechas > fin) | fechas.isna()

    dfs_22_23[nombre] = df.loc[mask_keep].reset_index(drop=True)

### Datos 2021-2022

In [46]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('Totalizadores Planta de Cerveza 2021_2022.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2021_2022 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2021_2022[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])


--- Resumen de hojas y columnas ---
                         hoja  filas  columnas                                                                                                                                                                                         nombres_columnas
0             Consolidado KPI  16049        62  DIA, HORA, EE Planta / Hl, EE Elaboracion / Hl, EE Bodega / Hl, EE Cocina / Hl, EE Envasado / Hl, EE Linea 2 / Hl, EE Linea 3 / Hl, EE Linea 4 / Hl, EE Linea 5 / Hl, EE Servicios / Hl, EE Sala Maq...
1      Consolidado Produccion  15573        12                DIA, HORA, Hl de Mosto, Hl Cerveza Cocina, Hl Producido Bodega, Hl Cerveza Filtrada, Hl Cerveza Envasada, Hl Cerveza L2, Hl Cerveza L3, Hl Cerveza L4, Hl Cerveza L5, Cocimientos Diarios
2    Totalizadores Produccion  15573        40  DIA, HORA, HL Mosto Budweiser, HL Mosto Tecate, HL Mosto Local, HL Mosto Heineken, HL Mosto Negra, HL Mosto Fuerte, HL Mosto Indio, HL Mosto Palermo, HL Mosto Bieckert, HL 

In [47]:
nombre_hoja_para_ver = 'Consolidado KPI'

if nombre_hoja_para_ver in dfs_2021_2022:
    
    print(f"  Mostrando la hoja: {nombre_hoja_para_ver}")
    
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 3 FILAS (.head()) ---")
    print(dfs_2021_2022[nombre_hoja_para_ver].head(3).to_string())
    
    print("\n\n--- ÚLTIMAS 3 FILAS (.tail()) ---")
    print(dfs_2021_2022[nombre_hoja_para_ver].tail(3).to_string())

else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs_2021_2022.keys()))

  Mostrando la hoja: Consolidado KPI

--- PRIMERAS 3 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl  Aire L4 / Hl  Aire L5 / Hl  Aire Se

In [48]:
print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs_2021_2022.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_lista = sorted(list(set(todos_los_dias) - set(dias_con_23_59)))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_lista = sorted(list(dias_con_horas_faltantes_sr.index))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_lista)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59                              dias_con_horas_faltantes
hoja                                                                                                                                                         
Consolidado KPI            2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13  4 días (Ej: 2022-03-16, 2022-07-01, 2022-07-13, ...)
Consolidado Produccion     2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13  4 días (Ej: 2022-03-16, 2022-07-01, 2022-07-13, ...)
Totalizadores Produccion   2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13  4 días (Ej: 2022-03-16, 2022-07-01, 2022-07-13, ...)
Consolidado EE             2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13                    2022-03-16, 2022-07-13, 2022-11-01
Totalizadore

In [49]:
dias_sin_23_59_lista

[datetime.date(2021, 7, 1),
 datetime.date(2022, 3, 2),
 datetime.date(2022, 3, 16),
 datetime.date(2022, 7, 13)]

In [50]:
if 'dfs_2021_2022' not in globals() or not isinstance(dfs_2021_2022, dict) or not dfs_2021_2022:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs_2021_2022.keys()):
        df = dfs_2021_2022[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs_2021_2022.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado Agua           2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado Aire           2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado EE             2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado GasVapor       2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado KPI            2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado Produccion     2021-01-01  2022-12-30  111 día

In [51]:
dias_faltantes_lista

[datetime.date(2021, 3, 31),
 datetime.date(2021, 5, 31),
 datetime.date(2021, 10, 31),
 datetime.date(2021, 12, 31),
 datetime.date(2022, 3, 17),
 datetime.date(2022, 3, 18),
 datetime.date(2022, 3, 19),
 datetime.date(2022, 3, 20),
 datetime.date(2022, 3, 21),
 datetime.date(2022, 3, 22),
 datetime.date(2022, 3, 23),
 datetime.date(2022, 3, 24),
 datetime.date(2022, 3, 25),
 datetime.date(2022, 3, 26),
 datetime.date(2022, 3, 27),
 datetime.date(2022, 3, 28),
 datetime.date(2022, 3, 29),
 datetime.date(2022, 3, 30),
 datetime.date(2022, 3, 31),
 datetime.date(2022, 4, 1),
 datetime.date(2022, 4, 2),
 datetime.date(2022, 4, 3),
 datetime.date(2022, 4, 4),
 datetime.date(2022, 4, 5),
 datetime.date(2022, 4, 6),
 datetime.date(2022, 4, 7),
 datetime.date(2022, 4, 8),
 datetime.date(2022, 4, 9),
 datetime.date(2022, 4, 10),
 datetime.date(2022, 4, 11),
 datetime.date(2022, 4, 12),
 datetime.date(2022, 4, 13),
 datetime.date(2022, 4, 14),
 datetime.date(2022, 4, 15),
 datetime.date(2022, 

In [52]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

dfs_21_22 = {}
hojas_saltadas = []

for hoja, df in dfs_2021_2022.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)

    # Filtramos filas sin día y agregamos orden determinístico
    tmp = tmp.dropna(subset=["_dia"]).copy()
    if tmp.empty:
        hojas_saltadas.append((hoja, "Sin días válidos"))
        continue

    tmp["_ord"] = np.arange(len(tmp))  # <- evita usar el índice en sort_values

    # Orden por día, hora (minutos) y orden original
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # Última fila por día (la mayor "_mins"; si empata, la última por "_ord")
    ultimas = tmp.groupby("_dia", as_index=False, sort=True).tail(1)

    # Limpieza de columnas auxiliares y orden final
    ultimas = ultimas.drop(columns=["_dia", "_mins", "_ord"]).sort_values(DAY_COL).reset_index(drop=True)

    dfs_21_22[hoja] = ultimas

In [53]:
dfk = dfs_21_22["Consolidado EE"].copy()

dfk["_hora_dt"] = pd.to_datetime(dfk["HORA"], errors="coerce")

mask_no_2359 = ~(
    (dfk["_hora_dt"].dt.hour == 23) &
    (dfk["_hora_dt"].dt.minute == 59))

df_no_2359 = dfk[mask_no_2359].drop(columns=["_hora_dt"])

# Ver resultados
print(len(df_no_2359), "filas con última hora distinta de 23:59")
print(df_no_2359["HORA"].value_counts(dropna=False).head())
df_no_2359.head(4)

3 filas con última hora distinta de 23:59
HORA
23:00:00    2
07:00:00    1
Name: count, dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14256\2004643425.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfk["_hora_dt"] = pd.to_datetime(dfk["HORA"], errors="coerce")


,DIA,HORA,Planta (Kw),Elaboracion (Kw),Bodega (Kw),Cocina (Kw),Envasado (Kw),Linea 2 (Kw),Linea 3 (Kw),Linea 4 (Kw),Servicios (Kw),Sala Maq (Kw),Aire (Kw),Calderas (Kw),Efluentes (Kw),Frio (Kw),Pta Agua / Eflu (Kw),Prod Agua (Kw),Resto Serv (Kw),Restos Planta (Kw),KW Gral Planta
421,2022-03-02,23:00:00,50076.11,7775.0,6118.0,2047.0,9258.0,2034.86,1766.0,6652.0,31239.0,20287.0,5390.0,396.0,1029.0,19294.0,1548.0,459.0,4078.0,1804.11,51549.0
435,2022-03-16,07:00:00,20493.18,2396.5,1937.5,603.0,5623.0,1778.68,2322.0,2412.0,10763.0,7326.0,2259.0,175.0,254.0,6292.0,459.0,191.0,1477.0,1710.68,20932.0
448,2022-07-13,23:00:00,72768.88,6938.5,5845.5,1901.0,17542.0,6024.13,6683.0,7423.0,49265.0,37052.0,7059.0,827.0,781.0,33704.0,1809.0,920.0,4594.0,-976.62,76246.0


In [54]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

# Aplicarlo a TODO el diccionario (una hoja por vez)
for nombre, df in dfs_21_22.items():
    dfs_21_22[nombre] = completar_e_interpolar_diario(df)

In [55]:
DAY_COL = "DIA"
inicio  = pd.Timestamp("2022-03-17")
fin     = pd.Timestamp("2022-06-30")

for nombre, df in dfs_21_22.items():
    if DAY_COL not in df.columns or df.empty:
        continue

    # Normalizar a fecha y construir máscara para CONSERVAR lo que queda fuera del rango
    fechas = pd.to_datetime(df[DAY_COL], dayfirst=True, errors="coerce").dt.normalize()

    # Rango INCLUSIVO: elimina 31/12/2023 ... 30/06/2024
    mask_keep = (fechas < inicio) | (fechas > fin) | fechas.isna()

    dfs_21_22[nombre] = df.loc[mask_keep].reset_index(drop=True)

### Unificación de los datos

Podemos ver que los datos comparten muchos días, por lo que son datos duplicados. Por eso, vamos a crear un solo diccionario que tenga todos los datos ordenados cronológicamente una sola vez.

In [61]:
import pandas as pd
from collections import defaultdict

DAY_COL  = "DIA"
HOUR_COL = "HORA"


RANGOS = {
    "dfs_21_22": [("2021-01-01", "2021-12-31")],
    "dfs_22_23": [("2022-01-01", "2022-12-31")],
    "dfs_23_24": [("2023-01-01", "2023-12-30"),
                  ("2024-07-01", "2024-10-26")],
}

HOJAS_INCLUIR = [
    "Consolidado KPI", "Consolidado Produccion", "Totalizadores Produccion", "Consolidado EE", "Totalizadores Energia",
    "Consolidado Agua", "Totalizadores Agua", "Consolidado GasVapor", "Totalizadores Gas y Vapor", "Consolidado Aire",
    "Totalizadores Aire", "Totalizadores Efluentes", "Totalizadores Glicol", "Totalizadores CO2"
]

# Tus diccionarios reales
DICS = {
    "dfs_21_22": dfs_21_22,
    "dfs_22_23": dfs_22_23,
    "dfs_23_24": dfs_23_24,
}


def slice_por_fecha(df, start, end, day_col=DAY_COL):
    if df.empty or day_col not in df.columns:
        return df.iloc[0:0]
    fechas = pd.to_datetime(df[day_col], errors="coerce", dayfirst=True).dt.normalize()
    mask = fechas.between(pd.to_datetime(start), pd.to_datetime(end), inclusive="both")
    return df.loc[mask].copy()

def ordenar_crono(df, day_col=DAY_COL, hour_col=HOUR_COL):
    if df.empty:
        return df
    dia = pd.to_datetime(df[day_col], errors="coerce", dayfirst=True)
    if hour_col in df.columns:
        dt = pd.to_datetime(dia.dt.date.astype(str) + " " + df[hour_col].astype(str),
                            errors="coerce", dayfirst=True)
    else:
        dt = dia
    return (df.assign(_dt=dt)
              .sort_values("_dt", kind="stable", na_position="last")
              .drop(columns="_dt").reset_index(drop=True))


partes_por_hoja = defaultdict(list)

for nombre_dic, dic in DICS.items():
    rangos = RANGOS.get(nombre_dic, [])
    for (inicio, fin) in rangos:
        for hoja, df in dic.items():
            if HOJAS_INCLUIR and hoja not in HOJAS_INCLUIR:
                continue
            recorte = slice_por_fecha(df, inicio, fin)
            if not recorte.empty:
                partes_por_hoja[hoja].append(recorte)

dfs_completo = {}
for hoja, partes in partes_por_hoja.items():
    # Unificar columnas: las faltantes se completan con 0
    todas_cols = list(set().union(*(p.columns for p in partes)))
    partes_alineadas = [p.reindex(columns=todas_cols, fill_value=0) for p in partes]
    combinado = pd.concat(partes_alineadas, ignore_index=True, sort=False)

    # Orden temporal final
    if DAY_COL in combinado.columns:
        combinado = ordenar_crono(combinado, DAY_COL, HOUR_COL)
    dfs_completo[hoja] = combinado